### Library import

In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import glob
import os

from string import punctuation
from nltk.corpus import stopwords
from collections import defaultdict, Counter
from wordcloud import WordCloud

!pip install textblob
from textblob import TextBlob

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.lda_model
import pyLDAvis.gensim_models

from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import LdaModel

import warnings
warnings.filterwarnings('ignore')

/Users/viviando/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/viviando/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/var/folders/b8/4ntn3_wd1wg59r0lmbfgfmwc0000gn/T/ipykernel_18686/3029400822.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import panda

/opt/miniconda3/envs/ADS500B/lib/python3.10/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [ ]:
# define function to display model topics and associated top words

def display_topics(model, features, no_top_words=5):
    for topic, words in enumerate(model.components_):
        total = words.sum()
        largest = words.argsort()[::-1] # invert sort order
        print("\nTopic %02d" % topic)
        for i in range(0, no_top_words):
            print("  %s (%2.2f)" % (features[largest[i]], abs(words[largest[i]]*100.0/total)))

In [3]:
df = pd.read_csv('data/news_cleaned.csv')
#df['tokens'] = df['tokens'].str.replace("'", "")
#df['tokens_no_climate'] = df['tokens_no_climate'].str.replace("'", "")
df.head(5)

,matchdatetime,station,snippet,tokens,snippet_no_climate,tokens_no_climate
0,2011-03-15 15:11:06,FOXNEWS,forward. greg: i suppose worth pointing out th...,"['forward', 'greg', 'suppose', 'worth', 'point...",forward. greg: i suppose worth pointing out th...,"['forward', 'greg', 'suppose', 'worth', 'point..."
1,2011-03-31 13:16:41,FOXNEWS,threaten a government shutdown. that's what's ...,"['threaten', 'government', 'shutdown', 'thats'...",threaten a government shutdown. that's what's ...,"['threaten', 'government', 'shutdown', 'thats'..."
2,2011-03-30 17:29:04,FOXNEWS,"and less likely. in any case, president obama ...","['less', 'likely', 'case', 'president', 'obama...","and less likely. in any case, president obama ...","['less', 'likely', 'case', 'president', 'obama..."
3,2011-03-14 22:09:55,FOXNEWS,environmental catastrophe in another part of t...,"['environmental', 'catastrophe', 'another', 'p...",environmental catastrophe in another part of t...,"['environmental', 'catastrophe', 'another', 'p..."
4,2011-03-21 19:03:10,FOXNEWS,"century: energy. to have clean, safe, renewabl...","['century', 'energy', 'clean', 'safe', 'renewa...","century: energy. to have clean, safe, renewabl...","['century', 'energy', 'clean', 'safe', 'renewa..."


In [34]:
# drop row where snippet only contains the following phrase:'climate change', 
# resulting in a null value for 'snippet_no_climate' column 

na_snippet = df[df['snippet_no_climate'].isna()]
df = df.drop(na_snippet.index)

### Topic modeling


Topic modeling was performed with the following iterations:
* `lda_model` uses unigrams, climate phrases included 
* `lda_model1` uses unigrams, climate phrases included 
* `lda_model2` uses bi-grams, climate phrases excluded 
* `lda_model3` uses bi-grams, climate phrases excluded 
* `lda_model4` uses unigrams and bi-grams, climate phrases included
* `lda_model5` uses unigrams and bi-grams, climate phrases excluded

In [ ]:
# Unigrams, climate phrases included
#combined_tokens=[token for sublist in df['tokens'] for token in sublist.split()]

count_vectorizer = CountVectorizer(stop_words='english', min_df=5, max_df=0.7)
count_vectors = count_vectorizer.fit_transform(df['snippet'])

lda_model = LatentDirichletAllocation(n_components=5, random_state=314)
W_lda_matrix = lda_model.fit_transform(count_vectors)
H_lda_matrix = lda_model.components_

display_topics(lda_model, count_vectorizer.get_feature_names_out())


Topic 00
  say (2.55)
  world (2.38)
  dont (2.14)
  know (2.11)
  right (1.78)

Topic 01
  warming (6.01)
  president (4.00)
  new (2.44)
  greenhouse (1.69)
  time (1.54)

Topic 02
  change (18.71)
  global (5.74)
  like (2.19)
  says (1.16)
  really (1.15)

Topic 03
  climate (21.08)
  think (2.87)
  thats (1.89)
  said (1.69)
  trump (1.60)

Topic 04
  people (3.61)
  going (3.38)
  issue (1.67)
  years (1.59)
  issues (1.35)


In [24]:
# Bigrams, climate phrases included

count_vectorizer1 = CountVectorizer(stop_words='english', min_df=5, max_df=0.7, ngram_range=(2,2))
count_vectors1 = count_vectorizer1.fit_transform(df['snippet'])

lda_model1 = LatentDirichletAllocation(n_components=5, random_state=314)
W_lda_matrix1 = lda_model1.fit_transform(count_vectors1)
H_lda_matrix1 = lda_model1.components_

display_topics(lda_model1, count_vectorizer1.get_feature_names_out())


Topic 00
  president trump (0.88)
  greenhouse gas (0.87)
  gas emissions (0.73)
  paris climate (0.64)
  global warming (0.60)

Topic 01
  global warming (2.29)
  health care (0.95)
  al gore (0.75)
  donald trump (0.31)
  white house (0.29)

Topic 02
  global warming (5.89)
  greenhouse gases (1.05)
  united states (0.66)
  greenhouse gas (0.52)
  gas emissions (0.40)

Topic 03
  global warming (1.36)
  carbon tax (0.63)
  change real (0.44)
  believe climate (0.38)
  don know (0.38)

Topic 04
  global warming (0.83)
  climate crisis (0.58)
  president obama (0.52)
  donald trump (0.39)
  change hoax (0.34)


In [53]:
# Unigrams, climate phrases excluded
combined_tokens_no_climate=[token for sublist in df['tokens_no_climate'] for token in sublist.split()]

count_vectorizer2 = CountVectorizer(stop_words='english', min_df=5, max_df=0.7)
count_vectors2 = count_vectorizer2.fit_transform(combined_tokens_no_climate)

lda_model2 = LatentDirichletAllocation(n_components=5, random_state=314)
W_lda_matrix2 = lda_model2.fit_transform(count_vectors2)
H_lda_matrix2 = lda_model2.components_

display_topics(lda_model2, count_vectorizer2.get_feature_names_out())



Topic 00
  say (2.82)
  new (2.64)
  dont (2.36)
  way (1.42)
  lot (1.41)

Topic 01
  people (3.88)
  thats (2.38)
  trump (2.01)
  want (1.83)
  issue (1.79)

Topic 02
  president (4.29)
  right (1.96)
  says (1.50)
  global (1.39)
  look (1.38)

Topic 03
  going (3.26)
  think (3.25)
  like (2.55)
  world (2.35)
  said (1.92)

Topic 04
  climate (2.39)
  know (2.19)
  years (1.61)
  time (1.56)
  emissions (1.46)


In [35]:
# Bigrams, climate phrases excluded, max_df=0.7

count_vectorizer3 = CountVectorizer(stop_words='english', min_df=5, max_df=0.7, ngram_range=(2,2))
count_vectors3 = count_vectorizer3.fit_transform(df['snippet_no_climate'])

lda_model3 = LatentDirichletAllocation(n_components=5, random_state=314)
W_lda_matrix3 = lda_model3.fit_transform(count_vectors3)
H_lda_matrix3 = lda_model3.components_

display_topics(lda_model3, count_vectorizer3.get_feature_names_out())


Topic 00
  new york (1.05)
  health care (0.96)
  don think (0.38)
  don know (0.34)
  cap trade (0.33)

Topic 01
  president obama (1.68)
  white house (0.88)
  president trump (0.36)
  gun control (0.29)
  donald trump (0.23)

Topic 02
  united states (1.80)
  donald trump (0.71)
  president trump (0.65)
  paris agreement (0.56)
  new deal (0.50)

Topic 03
  al gore (1.21)
  national security (0.47)
  going touched (0.42)
  species going (0.30)
  impacted species (0.30)

Topic 04
  president trump (0.63)
  sea level (0.27)
  nancy pelosi (0.23)
  years ago (0.23)
  donald trump (0.21)


In [45]:
# Unigrams and Bigrams, climate phrases included

count_vectorizer4 = CountVectorizer(stop_words='english', min_df=5, max_df=0.7, ngram_range=(1,2))
count_vectors4 = count_vectorizer4.fit_transform(df['snippet'])

lda_model4 = LatentDirichletAllocation(n_components=5, random_state=314)
W_lda_matrix4 = lda_model4.fit_transform(count_vectors4)
H_lda_matrix4 = lda_model4.components_

display_topics(lda_model4, count_vectorizer4.get_feature_names_out())


Topic 00
  global (1.93)
  warming (1.74)
  global warming (1.55)
  world (0.62)
  say (0.57)

Topic 01
  president (1.45)
  obama (0.76)
  new (0.56)
  president obama (0.50)
  world (0.47)

Topic 02
  think (1.13)
  people (0.91)
  president (0.81)
  going (0.79)
  like (0.78)

Topic 03
  global (2.85)
  warming (2.66)
  global warming (2.59)
  going (0.61)
  people (0.59)

Topic 04
  greenhouse (1.29)
  emissions (1.10)
  gas (0.96)
  carbon (0.89)
  greenhouse gas (0.71)


In [55]:
# Unigrams and Bigrams, climate phrases excluded

count_vectorizer5 = CountVectorizer(stop_words='english', min_df=5, max_df=0.7, ngram_range=(1,2))
count_vectors5 = count_vectorizer5.fit_transform(df['snippet_no_climate'])

lda_model5 = LatentDirichletAllocation(n_components=5, random_state=314)
W_lda_matrix5 = lda_model5.fit_transform(count_vectors5)
H_lda_matrix5 = lda_model5.components_

display_topics(lda_model5, count_vectorizer5.get_feature_names_out())


Topic 00
  think (1.12)
  president (0.95)
  people (0.83)
  like (0.79)
  going (0.78)

Topic 01
  people (0.79)
  new (0.74)
  said (0.55)
  climate (0.53)
  don (0.45)

Topic 02
  president (1.64)
  trump (0.88)
  world (0.85)
  paris (0.72)
  united (0.66)

Topic 03
  people (0.60)
  know (0.57)
  going (0.53)
  weather (0.52)
  just (0.50)

Topic 04
  world (0.79)
  emissions (0.71)
  carbon (0.68)
  scientists (0.63)
  report (0.56)


In [46]:
lda_display = pyLDAvis.lda_model.prepare(lda_model, count_vectors, count_vectorizer, sort_topics=False)
pyLDAvis.display(lda_display)

/Users/viviando/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/viviando/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/viviando/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/viviando/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pan

In [47]:
lda_display1 = pyLDAvis.lda_model.prepare(lda_model1, count_vectors1, count_vectorizer1, sort_topics=False)
pyLDAvis.display(lda_display1)

In [54]:
lda_display2 = pyLDAvis.lda_model.prepare(lda_model2, count_vectors2, count_vectorizer2, sort_topics=False)
pyLDAvis.display(lda_display2)

/Users/viviando/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/viviando/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/viviando/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/viviando/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pan

In [49]:
lda_display3 = pyLDAvis.lda_model.prepare(lda_model3, count_vectors3, count_vectorizer3, sort_topics=False)
pyLDAvis.display(lda_display3)

In [50]:
lda_display4 = pyLDAvis.lda_model.prepare(lda_model4, count_vectors4, count_vectorizer4, sort_topics=False)
pyLDAvis.display(lda_display4)

In [56]:
lda_display5 = pyLDAvis.lda_model.prepare(lda_model5, count_vectors5, count_vectorizer5, sort_topics=False)
pyLDAvis.display(lda_display5)

/Users/viviando/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/viviando/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/viviando/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/viviando/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pan